--------------------------------------------------------------------------------

# Created by [Nikolaos G. Giakoumoglou](https://github.com/giakou4)

### Electrical and Computer Engineer

Mobile: +30 6906430995 | e-mail: [ngiakoumoglou@gmail.gr](mailto:ngiakoumoglou@gmail.gr) | e-mail: [ngiakoumoglou@hotmail.gr](mailto:ngiakoumoglou@hotmail.gr) |

-------------------------------------------------------------------------------

# TensorFlow 2.x

This tutorial is based on [TensorFlow 2.x Object Detection API](https://blog.tensorflow.org/2020/07/tensorflow-2-meets-object-detection-api.html).


# 1 Install Dependencies

In [ ]:
%%capture
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%capture
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
%%capture
!pip install pycocotools
!pip install opencv-python-headless==4.5.2.52
!pip install -q roboflow

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import os, sys
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder
%matplotlib inline

print("Tensorflow version:", tf.__version__)
print("User Python Version:-", sys.version)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# 2 Download Dataset 
We'll download our dataset from [Roboflow](https://roboflow.com/).

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="QHFIRMdv15PvwRMvciev")
project = rf.workspace("team-roboflow").project("blood-cell-detection-1ekwu")
dataset = project.version(2).download("tfrecord")

test_record_fname = dataset.location + '/test/cells.tfrecord'
train_record_fname = dataset.location + '/train/cells.tfrecord'
label_map_pbtxt_fname = dataset.location + '/train/cells_label_map.pbtxt'

# 3 Train Custom TensorFlow Detector


## 3.1 Model Zoo

In [ ]:
#@title 
MODELS_CONFIG = {
        # ---------------------- CenterNet -------------------------------------
    'CenterNet HourGlass104 512x512': {
        'model_name': 'centernet_hourglass104_512x512_coco17_tpu-8',
        'base_pipeline_file': 'centernet_hg104_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'centernet_hg104_512x512_coco17_tpu-8.tar.gz',
        'batch_size': 2
    },
    'CenterNet HourGlass104 1024x1024': {
        'model_name': 'centernet_hourglass104_1024x1024_coco17_tpu-32',
        'base_pipeline_file': 'centernet_hg104_1024x1024_coco17_tpu-32.config',
        'pretrained_checkpoint': 'centernet_hg104_1024x1024_coco17_tpu-32.tar.gz',
        'batch_size': 1
    },
    'CenterNet Resnet50 V1 FPN 512x512': {
        'model_name': 'centernet_resnet50_v1_fpn_512x512_coco17_tpu-8',
        'base_pipeline_file': 'centernet_resnet50_v1_fpn_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'centernet_resnet50_v1_fpn_512x512_coco17_tpu-8.tar.gz',
        'batch_size': 2
    },
    'CenterNet Resnet101 V1 FPN 512x512': {
        'model_name': 'centernet_resnet101_v1_fpn_512x512_coco17_tpu-8',
        'base_pipeline_file': 'centernet_resnet101_v1_fpn_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'centernet_resnet101_v1_fpn_512x512_coco17_tpu-8.tar.gz',
        'batch_size': 2
    },
    'CenterNet Resnet50 V2 512x512': {
        'model_name': 'centernet_resnet50_v2_512x512_coco17_tpu-8',
        'base_pipeline_file': 'centernet_resnet50_v2_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'centernet_resnet50_v2_512x512_coco17_tpu-8.tar.gz',
        'batch_size': 2
    },
    'CenterNet MobileNetV2 FPN 512x512': {
        'model_name': 'centernet_mobilenetv2fpn_512x512_coco17_od',
        'base_pipeline_file': 'centernet_mobilenetv2fpn_512x512_coco17_od.config',
        'pretrained_checkpoint': 'centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz',
        'batch_size': 2
    },
        # ---------------------- EfficientDet ----------------------------------
    'EfficientDet D0 512x512': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 2
    },
    'EfficientDet D1 640x640': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 2
    },
    'EfficientDet D2 768x768': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 2
    },
    'EfficientDet D3 896x896': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 1
    },
    'EfficientDet D4 1024x1024': {
        'model_name': 'efficientdet_d4_coco17_tpu-32',
        'base_pipeline_file': 'efficientdet_d4_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d4_coco17_tpu-32.tar.gz',
        'batch_size': 1
    },
    'EfficientDet D5 1280x1280': {
        'model_name': 'efficientdet_d5_coco17_tpu-32',
        'base_pipeline_file': 'efficientdet_d5_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d5_coco17_tpu-32.tar.gz',
        'batch_size': 1
    },
    'EfficientDet D6 1280x1280': {
        'model_name': 'efficientdet_d6_coco17_tpu-32',
        'base_pipeline_file': 'efficientdet_d6_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d6_coco17_tpu-32.tar.gz',
        'batch_size': 1
    },
    'EfficientDet D7 1536x1536': {
        'model_name': 'efficientdet_d7_coco17_tpu-32',
        'base_pipeline_file': 'efficientdet_d7_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d7_coco17_tpu-32.tar.gz',
        'batch_size': 1
    },
        # ---------------------- SSD -------------------------------------------
    'SSD MobileNet v2 320x320': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
        'batch_size': 4
    },
    'SSD MobileNet V1 FPN 640x640': {
        'model_name': 'ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz',
        'batch_size': 2
    },
    'SSD MobileNet V2 FPNLite 320x320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
        'batch_size': 4
    },
    'SSD MobileNet V2 FPNLite 640x640': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz',
        'batch_size': 2
    },
    'SSD ResNet50 V1 FPN 640x640 (RetinaNet50)': {
        'model_name': 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8',
        'base_pipeline_file': 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz',
        'batch_size': 2
    },
    'SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)': {
        'model_name': 'ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8',
        'base_pipeline_file': 'ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8.tar.gz',
        'batch_size': 1
    },
    'SSD ResNet101 V1 FPN 640x640 (RetinaNet101)': {
        'model_name': 'ssd_resnet101_v1_fpn_640x640_coco17_tpu-8',
        'base_pipeline_file': 'ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz',
        'batch_size': 2
    },
    'SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)': {
        'model_name': 'ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8',
        'base_pipeline_file': 'ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8.tar.gz',
        'batch_size': 1
    },
    'SSD ResNet152 V1 FPN 640x640 (RetinaNet152)': {
        'model_name': 'ssd_resnet152_v1_fpn_640x640_coco17_tpu-8',
        'base_pipeline_file': 'ssd_resnet152_v1_fpn_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_resnet152_v1_fpn_640x640_coco17_tpu-8.tar.gz',
        'batch_size': 2
    },
    'SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)': {
        'model_name': 'ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8',
        'base_pipeline_file': 'ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8.tar.gz',
        'batch_size': 1
    },
        # ---------------------- Faster R-CNN ----------------------------------
    'Faster R-CNN ResNet50 V1 640x640': {
        'model_name': 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8',
        'base_pipeline_file': 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz',
        'batch_size': 2
    },
    'Faster R-CNN ResNet50 V1 1024x1024': {
        'model_name': 'faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8',
        'base_pipeline_file': 'faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8.config',
        'pretrained_checkpoint': 'faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8.tar.gz',
        'batch_size': 1
    },
    'Faster R-CNN ResNet50 V1 800x1333': {
        'model_name': 'faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8',
        'base_pipeline_file': 'faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8.config',
        'pretrained_checkpoint': 'faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8.tar.gz',
        'batch_size': 1
    },
    'Faster R-CNN ResNet101 V1 640x640': {
        'model_name': 'faster_rcnn_resnet101_v1_640x640_coco17_tpu-8',
        'base_pipeline_file': 'faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz',
        'batch_size': 2
    },
    'Faster R-CNN ResNet101 V1 1024x1024': {
        'model_name': 'faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8',
        'base_pipeline_file': 'faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.config',
        'pretrained_checkpoint': 'faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.tar.gz',
        'batch_size': 1
    },
    'Faster R-CNN ResNet101 V1 800x1333': {
        'model_name': 'faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8',
        'base_pipeline_file': 'faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8.config',
        'pretrained_checkpoint': 'faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8.tar.gz',
        'batch_size': 1
    },
    'Faster R-CNN ResNet152 V1 640x640': {
        'model_name': 'faster_rcnn_resnet152_v1_640x640_coco17_tpu-8',
        'base_pipeline_file': 'faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz',
        'batch_size': 1
    },
    'Faster R-CNN ResNet152 V1 1024x1024': {
        'model_name': 'faster_rcnn_resnet152_v1_1024x1024_coco17_tpu-8',
        'base_pipeline_file': 'faster_rcnn_resnet152_v1_1024x1024_coco17_tpu-8.config',
        'pretrained_checkpoint': 'faster_rcnn_resnet152_v1_1024x1024_coco17_tpu-8.tar.gz',
        'batch_size': 1
    },
    'Faster R-CNN ResNet152 V1 800x1333': {
        'model_name': 'faster_rcnn_resnet152_v1_800x1333_coco17_gpu-8',
        'base_pipeline_file': 'faster_rcnn_resnet152_v1_800x1333_coco17_gpu-8.config',
        'pretrained_checkpoint': 'faster_rcnn_resnet152_v1_800x1333_coco17_gpu-8.tar.gz',
        'batch_size': 1
    },
    'Faster R-CNN Inception ResNet V2 640x640': {
        'model_name': 'faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8',
        'base_pipeline_file': 'faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz',
        'batch_size': 1
    },
    'Faster R-CNN Inception ResNet V2 1024x1024': {
        'model_name': 'faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8',
        'base_pipeline_file': 'faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.config',
        'pretrained_checkpoint': 'faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz',
        'batch_size': 1
    },
    # ---------------------- Mask R-CNN ----------------------------------------
    'Mask R-CNN Inception ResNet V2 1024x1024': {
        'model_name': 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8',
        'base_pipeline_file': 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config',
        'pretrained_checkpoint': 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz',
        'batch_size': 1
    },
}

## 3.2 Choose model

In [ ]:
chosen_model = 'Faster R-CNN ResNet50 V1 640x640' #@param = ['CenterNet HourGlass104 512x512', 'CenterNet HourGlass104 1024x1024', 'CenterNet Resnet50 V1 FPN 512x512', 'CenterNet Resnet101 V1 FPN 512x512', 'CenterNet Resnet50 V2 512x512',  'CenterNet MobileNetV2 FPN 512x512', 'EfficientDet D0 512x512', 'EfficientDet D1 640x640', 'EfficientDet D2 768x768', 'EfficientDet D3 896x896', 'EfficientDet D4 1024x1024', 'EfficientDet D5 1280x1280', 'EfficientDet D6 1280x1280', 'EfficientDet D7 1536x1536', 'SSD MobileNet v2 320x320', 'SSD MobileNet V1 FPN 640x640', 'SSD MobileNet V2 FPNLite 320x320', 'SSD MobileNet V2 FPNLite 640x640',  'SSD ResNet50 V1 FPN 640x640 (RetinaNet50)', 'SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)', 'SSD ResNet101 V1 FPN 640x640 (RetinaNet101)', 'SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)', 'SSD ResNet152 V1 FPN 640x640 (RetinaNet152)', 'SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)', 'Faster R-CNN ResNet50 V1 640x640', 'Faster R-CNN ResNet50 V1 1024x1024', 'Faster R-CNN ResNet50 V1 800x1333', 'Faster R-CNN ResNet101 V1 640x640', 'Faster R-CNN ResNet101 V1 1024x1024', 'Faster R-CNN ResNet101 V1 800x1333', 'Faster R-CNN ResNet152 V1 640x640', 'Faster R-CNN ResNet152 V1 1024x1024', 'Faster R-CNN ResNet152 V1 800x1333', 'Faster R-CNN Inception ResNet V2 640x640', 'Faster R-CNN Inception ResNet V2 1024x1024', 'Mask R-CNN Inception ResNet V2 1024x1024']
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

## 3.3 Download pre-trained weights
Pre-trained weights are availiable at http://download.tensorflow.org/models/object_detection/tf2/20200711/

In [ ]:
%%capture
%mkdir /content/deploy/
%cd /content/deploy/
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

## 3.4 Download configuration file

Configuration files are downloaded from [giakou4](https://github.com/giakou4)'s github

In [ ]:
%%capture
%cd /content/deploy
download_config = 'https://raw.githubusercontent.com/giakou4/tf2_object_detection_configs/master/configs/' + base_pipeline_file
!wget {download_config}

## 3.5 Alter configuration file

In [ ]:
num_steps = 200000 #@param {type:"integer"}
num_eval_steps = 1000 #@param {type:"integer"}
batch_size = 4 #@param {type:"integer"}

In [ ]:
#@title 3.5.1 Prepare configuration file
pipeline_fname = '/content/deploy/' + base_pipeline_file
fine_tune_checkpoint = '/content/deploy/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

num_classes = get_num_classes(label_map_pbtxt_fname)

In [ ]:
#@title 3.5.2 Write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file
%cd /content/deploy

import re

with open(pipeline_fname) as f:
    s = f.read()

with open('pipeline_file.config', 'w') as f:
    s = re.sub('fine_tune_checkpoint: ".*?"', 'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    s = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)
    s = re.sub('label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)
    s = re.sub('batch_size: [0-9]+', 'batch_size: {}'.format(batch_size), s)
    s = re.sub('num_steps: [0-9]+', 'num_steps: {}'.format(num_steps), s)
    s = re.sub('num_classes: [0-9]+', 'num_classes: {}'.format(num_classes), s)
    s = re.sub('fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
    f.write(s)

# 3.6 Train Detector

* ```pipeline_file```: defined above in writing custom training configuration (default: ```'/content/deploy/pipeline_file.config'```)
* ```model_dir```: the location tensorboard logs and saved model checkpoints will save to (default: ```'/content/training/'```)
* ```num_train_steps```: how long to train for (default: ```5000```)
* ```num_eval_steps```: perform eval on validation set after this many steps (default: ```1000```)

In [ ]:
pipeline_file = '/content/deploy/pipeline_file.config'
model_dir = '/content/training/'
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/training/train'

## 3.7 Export Trained Inference Graph

In [ ]:
%%capture
import re
import numpy as np

output_directory = '/content/fine_tuned_model'
last_model_path = '/content/training/'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

# 4 Validate Custom TensorFlow Detector 

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir} \
    --eval_timeout=180 \
# Not yet implemented for EfficientDet